In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [2]:
# transform = transforms.Compose([
#     transforms.Resize((32, 32)),
#     transforms.ToTensor(),
# ])

transform_train = transforms.Compose([
    transforms.Resize((32, 32)),  # Resize images to 64x64
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
])

transform_test = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
])

In [3]:
train_data = datasets.ImageFolder(root='Dataset/train', transform=transform_train)
test_data = datasets.ImageFolder(root='Dataset/test', transform=transform_test)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=True)

In [7]:
# class CNN(nn.Module):
#     def __init__(self):
#         super(CNN, self).__init__()
#         self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
#         self.relu = nn.ReLU()
#         self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
#         self.fc1 = nn.Linear(16 * 16 * 16, 128)  # Calculated for 32x32 input images
#         self.fc2 = nn.Linear(128, 2)  # 2 classes: fake and real

#     def forward(self, x):
#         x = self.pool(self.relu(self.conv1(x)))
#         x = x.view(-1, 16 * 16 * 16)  # Calculated for 32x32 input images
#         x = self.relu(self.fc1(x))
#         x = self.fc2(x)
#         return x

# model = CNN()

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 2)  # 2 classes: fake and real

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [8]:
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)

model = CNN()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [9]:
# num_epochs = 10
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model.to(device)

# for epoch in range(num_epochs):
#     model.train()
#     running_loss = 0.0
#     for images, labels in train_loader:
#         images, labels = images.to(device), labels.to(device)
#         optimizer.zero_grad()
#         outputs = model(images)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()
#         running_loss += loss.item()
    
#     print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}')


num_epochs = 15
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    train_accuracy = 100 * correct / total
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}, Train Acc: {train_accuracy:.2f}%')

Epoch 1/15, Loss: 0.3088, Train Acc: 86.39%
Epoch 2/15, Loss: 0.2083, Train Acc: 91.62%
Epoch 3/15, Loss: 0.1837, Train Acc: 92.64%
Epoch 4/15, Loss: 0.1664, Train Acc: 93.34%
Epoch 5/15, Loss: 0.1561, Train Acc: 93.89%
Epoch 6/15, Loss: 0.1458, Train Acc: 94.29%
Epoch 7/15, Loss: 0.1402, Train Acc: 94.43%
Epoch 8/15, Loss: 0.1348, Train Acc: 94.67%
Epoch 9/15, Loss: 0.1285, Train Acc: 94.93%
Epoch 10/15, Loss: 0.1229, Train Acc: 95.25%
Epoch 11/15, Loss: 0.1192, Train Acc: 95.34%
Epoch 12/15, Loss: 0.1128, Train Acc: 95.56%
Epoch 13/15, Loss: 0.1101, Train Acc: 95.78%
Epoch 14/15, Loss: 0.1066, Train Acc: 95.89%
Epoch 15/15, Loss: 0.1019, Train Acc: 96.02%


In [10]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Test Accuracy: {accuracy}%')

Test Accuracy: 95.505%
